In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i 'gcmc.py'

In [ ]:
filename = "output_data.csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.ycf[i]):
        yco = data_dump.yc0[i]
        P_res = data_dump.P0[i]*10**5 #[Pa]
        T = 45 + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5

        print( "Co:\t", round( fme/10**5, 1 ), "bar\tMe:\t", round( fme/10**5, 1 ), "bar")

        s_box = 50
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = floor(fco*Vol/kb/T)
        Nme = floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 1000
        N_equil = 1000
        N_prod = 1000

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.ycf[i] = Nco/(Nco + Nme)
        data_dump.Pf[i] = Pv.mean()*10 #[bar]
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.ycf_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.Pf_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.t_comp[i] = t - t0
        print( i, data_dump.ycf[i],  data_dump.Pf[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.t_comp[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

Co:	 68.7 bar	Me:	 68.7 bar
